In [1]:
# 뉴스 요약 코드 
## 뉴스 요약
### 모델 연결, 뉴스 본문 인식 후 간략히 요약

# 일자, 제목, 본문만 선택

In [9]:
# 자동으로 summary를 해주는 sumy library 설치
pip install sumy

import pandas as pd 
from sumy.parsers.plaintext import PlaintextParser  #텍스트 문서를 문장으로 분리
from sumy.nlp.tokenizers import Tokenizer           
from sumy.summarizers.lsa import LsaSummarizer

import nltk
nltk.download('punkt')

In [9]:
# CSV 파일에서 데이터프레임 생성
news = pd.read_csv("삼성 뉴스_(20220910-20230910).csv")

# 원하는 열 인덱스로 설정
news = news.set_index(['일자', '제목', '본문'])

# news라는 데이터프레임 출력
print(news)

                                                                                                               뉴스 식별자  \
일자       제목                                          본문                                                                 
20230910 다시 늘어난 '빚투' 간 큰 '도박개미' 몰리는 종목               신용거래융자 잔액이 석 달 새 2조원 가까이 늘어나며 20조원을 넘어섰다. 지난 4월...  2.100601e+06   
         '인재경영' 계승하는 이재용 삼성, 대규모 신입공채 돌입             4대 그룹 중 유일하게 공채를 유지하고 있는 삼성이 하반기 신입사원 공채에 들어간다....  2.100601e+06   
         삼성, 4대그룹 유일 하반기 공채... 대기업 10곳 중 6곳 "채용 불투명" 삼성전자 등 20개 관계사 채용 \n오는 18일까지 지원서 접수 \n \n \n4대 ...  2.100501e+06   
         대기업 절반이나 "채용계획 못세워"                         대기업 10곳 중 6곳은 하반기 신규 채용을 하지 않거나 아직 계획이 없는 것으로 나...  2.100101e+06   
         돌발질문에도 대답 척척  AI 만난 가상인간의 진화                "누군가에게 감동과 위로를 전달할 수 있는 예술가이자 긍정적인 영향력을 가진 친구가 ...  2.100101e+06   
...                                                                                                               ...   
20220910 애플 삼성 비켜 中저가 제품이 주도한 2Q

In [35]:
# 요약된 내용을 저장할 빈 리스트 생성함
summaries = []

# LsaSummarizer 모델 생성, summarizer라 이름 붙임
summarizer = LsaSummarizer()

# "본문" 열에 대한 요약 생성 및 저장
for index, row in news.iterrows():
    news_text = row['본문']
    
    # 본문이 비어있지 않은 경우에만 요약 생성하도록 설정
    if not pd.isnull(news_text):
        # PlaintextParser와 Tokenizer를 사용하여 문장 토큰화 시킴
        parser = PlaintextParser.from_string(news_text, Tokenizer("english"))
        
        # 요약 생성
        summary = summarizer(parser.document, sentences_count=3)  ## 원하는 문장 수를 지정 = 2-> 5 -> 2(우선은 2로 차후 문제 발생시 수정)
        summaries.append(" ".join([str(sentence) for sentence in summary]))
    else:
        summaries.append("")  # 본문이 비어있는 경우 빈 문자열 추가함

# 요약된 내용을 "요약" 열에 추가함
news["요약"] = summaries

# 요약된 데이터를 새로운 CSV 파일로 저장
news.to_csv("news_sumy.csv", index=False)

In [36]:
#요약 column이 포함된 news_sumy라는 새로운 뉴스 데이터를 불러옴
news_sumy = pd.read_csv("news_sumy.csv")
news_sumy.head()

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,사건/사고 분류3,인물,위치,기관,키워드,특성추출(가중치순 상위 50개),본문,URL,분석제외 여부,요약
0,2.100601e+06,20230910,한국경제,박의명,다시 늘어난 '빚투' 간 큰 '도박개미' 몰리는 종목,경제>유통,경제>금융_재테크,경제>부동산,NaN,NaN,NaN,NaN,NaN,"삼천리,한미글로벌,포스코퓨처엠,네이버,KODEX코스닥,서울가스,LS,금융투자협회,태...","빚투,도박개미,종목,신용,거래,융자,잔액,2조,20조,무더기,하한,사태,감소세,빚투...","빚투,코스닥,대형주,유가증권시장,신용잔액,인버스,포스코홀딩스,20조,투자자,증권사,...",신용거래융자 잔액이 석 달 새 2조원 가까이 늘어나며 20조원을 넘어섰다. 지난 4...,https://www.hankyung.com/article/2023091070761,NaN,지난 4월 무더기 하한가 사태 이후 감소세를 나타내던 ‘빚투’(빚내서 투자)가 다시...
1,2.100601e+06,20230910,한국경제,최예린,"'인재경영' 계승하는 이재용 삼성, 대규모 신입공채 돌입",경제>취업_창업,경제>반도체,경제>산업_기업,NaN,NaN,NaN,"이재용,이병철",경기,"유일,삼성전자,삼성,GSAT","인재경영,계승,이재용,삼성,돌입,신입공채,그룹,유일,공채,유지,삼성,신입,사원,공채...","삼성,간담회,신입사원,삼성전자,이재용,이병철,관계사,신입공채,인재경영,직군,스마트폰...",4대 그룹 중 유일하게 공채를 유지하고 있는 삼성이 하반기 신입사원 공채에 들어간다...,https://www.hankyung.com/article/2023091071401,NaN,4대 그룹 중 유일하게 공채를 유지하고 있는 삼성이 하반기 신입사원 공채에 들어간다...
2,2.100501e+06,20230910,파이낸셜뉴스,김준석 기자 (rejune1112@fnnews.com),"삼성, 4대그룹 유일 하반기 공채... 대기업 10곳 중 6곳 ""채용 불투명""",경제>취업_창업,경제>산업_기업,경제>반도체,사회>사회문제>성차별,NaN,NaN,"이재용,이,이병철","중국,경기,고금","유일,GSA,삼성디스플레이,전국경제인연합회,리서치앤리서치,삼성전자,삼성전기,삼성,삼...","삼성,그룹,유일,공채,불투명,채용,삼성전자,채용,관계사,18일,지원서,접수,그룹,유...","삼성,삼성전자,외국인,상반기,리서치앤리서치,이병철,지원서,온라인,매출액,스플레이,기...",삼성전자 등 20개 관계사 채용 \n오는 18일까지 지원서 접수 \n \n \n4대...,http://www.fnnews.com/news/202309101808193904,NaN,삼성전자 등 20개 관계사 채용 오는 18일까지 지원서 접수 삼성은 이병철 창업회장...
3,2.100101e+06,20230910,매일경제,"정승환 전문기자(fanny@mk.co.kr), 이새하 기자(ha12@mk.co.kr)","대기업 절반이나 ""채용계획 못세워""",경제>취업_창업,경제>유통,경제>반도체,NaN,NaN,NaN,NaN,NaN,"삼성물산,삼성SDS,삼성화재,유일,삼성디스플레이,전국경제인연합회,리서치앤리서치,삼성...","절반,채용계획,6곳,신규,채용,계획,취업,와중,삼성,그룹,유일,공채,시작,전국경제인...","삼성,매출액,기업들,경쟁률,10곳,6곳,실업률,일주일간,입사자",대기업 10곳 중 6곳은 하반기 신규 채용을 하지 않거나 아직 계획이 없는 것으로 ...,http://www.mk.co.kr/article/10826390,NaN,대기업 10곳 중 6곳은 하반기 신규 채용을 하지 않거나 아직 계획이 없는 것으로 ...
4,2.100101e+06,20230910,매일경제,"정호준(jeong.hojun@mk.co.kr), 원호섭 기자(wonc@mk.co.kr)",돌발질문에도 대답 척척 AI 만난 가상인간의 진화,경제>산업_기업,NaN,NaN,NaN,NaN,NaN,"정지형,김태용",NaN,"경영연구원,비브스튜디오스,YBM,스마일게이트,삼성전자,국방홍보원,삼성,유공,매일경제...","돌발질문,대답,AI,가상인간,진화,감동,위로,전달,예술가,긍정적,영향력,친구,스마일...","가상인간,스마일게이트,ai,레고,비플,삼일회계법인,예술가,인플루언서,비브스튜디오스,...","""누군가에게 감동과 위로를 전달할 수 있는 예술가이자 긍정적인 영향력을 가진 친구가...",http://www.mk.co.kr/article/10826395,NaN,"""누군가에게 감동과 위로를 전달할 수 있는 예술가이자 긍정적인 영향력을 가진 친구가..."


In [38]:
# news_sumy에서 '일자', '제목', '본문' , '요약' 열을 선택하여 news_sumy2라는 새로운 DataFrame 생성
news_sumy2 = news_sumy[['일자', '제목', '본문', '요약']]

news_sumy2.to_csv("news_sumy2.csv", index=False)
news_sumy2

,일자,제목,본문,요약
0,20230910,다시 늘어난 '빚투' 간 큰 '도박개미' 몰리는 종목,신용거래융자 잔액이 석 달 새 2조원 가까이 늘어나며 20조원을 넘어섰다. 지난 4...,지난 4월 무더기 하한가 사태 이후 감소세를 나타내던 ‘빚투’(빚내서 투자)가 다시...
1,20230910,"'인재경영' 계승하는 이재용 삼성, 대규모 신입공채 돌입",4대 그룹 중 유일하게 공채를 유지하고 있는 삼성이 하반기 신입사원 공채에 들어간다...,4대 그룹 중 유일하게 공채를 유지하고 있는 삼성이 하반기 신입사원 공채에 들어간다...
2,20230910,"삼성, 4대그룹 유일 하반기 공채... 대기업 10곳 중 6곳 ""채용 불투명""",삼성전자 등 20개 관계사 채용 \n오는 18일까지 지원서 접수 \n \n \n4대...,삼성전자 등 20개 관계사 채용 오는 18일까지 지원서 접수 삼성은 이병철 창업회장...
3,20230910,"대기업 절반이나 ""채용계획 못세워""",대기업 10곳 중 6곳은 하반기 신규 채용을 하지 않거나 아직 계획이 없는 것으로 ...,대기업 10곳 중 6곳은 하반기 신규 채용을 하지 않거나 아직 계획이 없는 것으로 ...
4,20230910,돌발질문에도 대답 척척 AI 만난 가상인간의 진화,"""누군가에게 감동과 위로를 전달할 수 있는 예술가이자 긍정적인 영향력을 가진 친구가...","""누군가에게 감동과 위로를 전달할 수 있는 예술가이자 긍정적인 영향력을 가진 친구가..."
...,...,...,...,...
15982,20220910,애플 삼성 비켜 中저가 제품이 주도한 2Q 무선 이어폰,[아시아경제 차민영 기자] 올해 2분기 전 세계 무선 이어폰 시장이 전년 대비 15...,9일 글로벌 시장조사업체 카운터포인트리서치가 최근 발표한 무선 이어폰 마켓 트래커에...
15983,20220910,삼성 '스마트싱스' 성공 이것에 달렸다,[아시아경제 문채석 기자] 삼성전자가 전기료와 에너지를 모두 절감하는 가전 플랫폼인...,[아시아경제 문채석 기자] 삼성전자가 전기료와 에너지를 모두 절감하는 가전 플랫폼인...
15984,20220910,규제 벗어난 전기차 폐배터리 공급망 문제 대안 급부상,[아시아경제 오현길 기자] 정부가 전기차 보급 확대로 급증할 것으로 예상되는 전기차...,[아시아경제 오현길 기자] 정부가 전기차 보급 확대로 급증할 것으로 예상되는 전기차...
15985,20220910,은행 하반기 채용문 열렸다,[헤럴드경제=박자연 기자]은행권이 하반기 채용을 시작했다. 시중은행부터 인터넷은행까...,"신입행원 외 경력직, 전문인력 채용도 동시에 진행된다. 8일 은행권은 시중은행을 필..."


In [39]:
news_sumy3 = news_sumy[['일자', '제목', '요약']]
# news_sumy3["요약"]   # 최종적으로 요약 항목이 어떻게 저장되있는지 확인 

0        지난 4월 무더기 하한가 사태 이후 감소세를 나타내던 ‘빚투’(빚내서 투자)가 다시...
1        4대 그룹 중 유일하게 공채를 유지하고 있는 삼성이 하반기 신입사원 공채에 들어간다...
2        삼성전자 등 20개 관계사 채용 오는 18일까지 지원서 접수 삼성은 이병철 창업회장...
3        대기업 10곳 중 6곳은 하반기 신규 채용을 하지 않거나 아직 계획이 없는 것으로 ...
4        "누군가에게 감동과 위로를 전달할 수 있는 예술가이자 긍정적인 영향력을 가진 친구가...
                               ...                        
15982    9일 글로벌 시장조사업체 카운터포인트리서치가 최근 발표한 무선 이어폰 마켓 트래커에...
15983    [아시아경제 문채석 기자] 삼성전자가 전기료와 에너지를 모두 절감하는 가전 플랫폼인...
15984    [아시아경제 오현길 기자] 정부가 전기차 보급 확대로 급증할 것으로 예상되는 전기차...
15985    신입행원 외 경력직, 전문인력 채용도 동시에 진행된다. 8일 은행권은 시중은행을 필...
15986    국내 재계 서열 1위인 삼성전자(005930)의 이재용 부회장이 추석 연휴를 기점으...
Name: 요약, Length: 15987, dtype: object

In [ ]:
# 이후 선택한 뉴스의 전체 본문 첨가 필요